In [5]:
# doc officielle de selenium: https://selenium-python.readthedocs.io/index.html
# lien pour installer le webdriver à coup sûr: https://chromedriver.chromium.org/getting-started
# selon la version de selenium, les webdriver ne sont pas compatibles avec google collab. 
# il faut enregister ce fichier dans le même dossier que chromedriver.exe

# !pip install selenium
# !pip install web_driver_manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
import os
import json
import urllib
import sys
import time

In [11]:
#on importe les driver et on se connecte à google
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")
options = webdriver.ChromeOptions()

##pour régler le problème d'interactibilité ??
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

# on implémente le path vers geckodriver vers la variable d'environnement OS
os.environ["PATH"] += os.pathsep + os.getcwd()


# Configuration
download_path = "dataset/"
# Images
words_to_search = ['Dépôt d\'ordures nature','nature champs']
nb_to_download = [30,30]
#number = nb_to_download[0]
first_image_position = [10,10]
#first_position = first_image_position[0]

def main():
    if len(words_to_search) != len(nb_to_download) or len(nb_to_download) != len(first_image_position) :
        raise ValueError('You may have forgotten to configure one of the lists (length is different)')
    i= 0
    # pour chaque mot dans la liste, on télécharge le nombre d'images
    while i<len(words_to_search):
        print("Words "+str(i)+" : "+str(nb_to_download[i])+"x\""+words_to_search[i]+"\"")
        if nb_to_download[i] > 0:
            search_and_save(words_to_search[i],nb_to_download[i], first_image_position[i])
        i+=1

def search_and_save(text, number, first_position):
    # Number_of_scrolls * 400 images will be opened in the browser
    number_of_scrolls = int((number + first_position)/ 400 + 1) 
    print("Search : "+text+" ; number : "+str(number)+"; first_position : "+str(first_position)+" ; scrolls : "+str(number_of_scrolls))

    # on créé les répertoires pour enregistrer les images
    if not os.path.exists(download_path + text.replace(" ", "_")):
        os.makedirs(download_path + text.replace(" ", "_"))

    # on se connecte à google image
    url = "https://www.google.co.in/search?q="+text+"&source=lnms&tbm=isch"
    driver = webdriver.Chrome()
    driver.get(url)
    # ne pas oublier de dire à notre script de cocher la case "tout accepter" concernant les cookies
    driver.find_element(By.XPATH, "//*[@id='yDmH0d']/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span").click()
    headers = {}
    headers['User-Agent'] = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
    extensions = {"jpg", "jpeg", "png", "gif"}
    
    img_count = 0
    downloaded_img_count = 0
    img_skip = 0
    
    # on prépare la page google
    for _ in range(number_of_scrolls):
        for __ in range(10):
            # on multiplie par le nombre de scrolls 
            driver.execute_script("window.scrollBy(0, 1000000)")
            time.sleep(0.2)
        # pour charger 400 images
        time.sleep(2.5)
        try:
            driver.find_element(By.XPATH, "/html/body/div[2]/c-wiz/div[3]/div[1]/div/div/div/div/div[1]/div[2]/div[2]/input").click()
            time.sleep(2.5)
        except Exception as e:
            print("Less images found:"+ str(e))
            break

    # on télécharge les images
    imges = driver.find_elements(By.XPATH, '/html/body/div[2]/c-wiz/div[3]/div[1]/div/div/div/div/div[1]/div[1]/span/div[1]/div[1]/div[1]/a[1]/div[1]/img')   
    print("Total images:"+ str(len(imges)) + "\n")
    for img in imges:
        if img_skip < first_position:
            # on saute les premières images si c'est demandé
            img_skip += 1
        else :
            # on récupère les images
            img_count += 1
            img_url = json.loads(img.get_attribute('innerHTML'))["ou"]
            img_type = json.loads(img.get_attribute('innerHTML'))["ity"]
            print("Downloading image "+ str(img_count) + ": "+ img_url)
            try:
                if img_type not in extensions:
                    img_type = "jpg"
                # on télécharge les images et on sauvegarde
                req = urllib.request.Request(img_url, headers=headers)
                raw_img = urllib.request.urlopen(req).read()
                f = open(download_path+text.replace(" ", "_")+"/"+str(img_skip+downloaded_img_count)+"."+img_type, "wb")
                f.write(raw_img)
                f.close
                downloaded_img_count += 1
            except Exception as e:
                print("Download failed:"+ str(e))
            finally:
                print("")
            if downloaded_img_count >= number:
                break

    print("Total skipped : "+str(img_skip)+"; Total downloaded : "+ str(downloaded_img_count)+ "/"+ str(img_count))
    driver.quit()

if __name__ == "__main__":
    main()

Words 0 : 30x"Dépôt d'ordures nature"
Search : Dépôt d'ordures nature ; number : 30; first_position : 10 ; scrolls : 1
Less images found:Message: element not interactable
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x00F5ACD3+2075859]
	Ordinal0 [0x00EEEE61+1633889]
	Ordinal0 [0x00DEB680+571008]
	Ordinal0 [0x00E1BBD6+768982]
	Ordinal0 [0x00E11C76+728182]
	Ordinal0 [0x00E3731C+881436]
	Ordinal0 [0x00E115BF+726463]
	Ordinal0 [0x00E37534+881972]
	Ordinal0 [0x00E4B56A+963946]
	Ordinal0 [0x00E37136+880950]
	Ordinal0 [0x00E0FEFD+720637]
	Ordinal0 [0x00E10F3F+724799]
	GetHandleVerifier [0x0120EED2+2769538]
	GetHandleVerifier [0x01200D95+2711877]
	GetHandleVerifier [0x00FEA03A+521194]
	GetHandleVerifier [0x00FE8DA0+516432]
	Ordinal0 [0x00EF682C+1665068]
	Ordinal0 [0x00EFB128+1683752]
	Ordinal0 [0x00EFB215+1683989]
	Ordinal0 [0x00F06484+1729668]
	BaseThreadInitThunk [0x7607FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x76FE7BBE+286]
	RtlGetAppContainerNamedObjectPat